<img src="https://www.mercari.com/assets/img/help_center/us/ogp.png"/>

# Mercari Price Suggestion Challenge
***
### Can you automatically suggest product prices to online sellers?

**Product pricing gets even harder at scale**, considering just how many products are sold online. Clothing has strong seasonal pricing trends and is heavily influenced by brand names, while electronics have fluctuating prices based on product specs.

**Mercari**, Japan’s biggest community-powered shopping app, knows this problem deeply. They’d like to offer pricing suggestions to sellers, but this is tough because their sellers are enabled to put just about anything, or any bundle of things, on Mercari's marketplace.

In this competition, Mercari’s challenging you to **build an algorithm that automatically suggests the right product prices**. You’ll be provided user-inputted text descriptions of their products, including details like product category name, brand name, and item condition.

### Dataset Features

- **ID**: the id of the listing
- **Name:** the title of the listing
- **Item Condition:** the condition of the items provided by the seller
- **Category Name:** category of the listing
- **Brand Name:** brand of the listing
- **Shipping:** whether or not shipping cost was provided
- **Item Description:** the full description of the item
- **Price:** the price that the item was sold for. This is the target variable that you will predict. The unit is USD.

### Key Words
- Pricing Recommendation
- Product Features
- NLP
- C2C & B2C

**Source:** https://www.kaggle.com/c/mercari-price-suggestion-challenge

<img src = "https://cdn.dribbble.com/users/56196/screenshots/2281553/mobile-dribbble.gif"/>

# Representing and Mining Text
***
Since, text is the most **unstructured** form of all the available data, various types of noise are present in it and the data is not readily analyzable without any pre-processing. The entire process of cleaning and standardization of text, making it noise-free and ready for analysis is known as **text pre-processing**.

### Fundamental Concepts 

The importance of constructing mining-friendly data representations; Representation of text for data mining. 

### Important Terminologies
- **Document**: One piece of text. It could be a single sentence, a paragraph, or even a full page report. 
- **Tokens**: Also known as terms. It is simply just a word. So many tokens form a document. 
- **Corpus**: A collection of documents. 
- **Term Frequency (TF)**: Measures how often a term is in a single document
- **Inverse Document Frequency (IDF)**: distribution of a term over a corpus

### Pre-Processing Techniques
- **Stop Word Removal:** stop words are terms that have little no meaning in a given text. Think of it as the "noise" of data. Such terms include the words, "the", "a", "an", "to", and etc...
- **Bag of Words Representation: ** treats each word as a feature of the document

- **TFIDF**: a common value representation of terms. It boosts or weighs words that have low occurences. For example, if the word "play" is common, then there is little to no boost. But if the word "mercari" is rare, then it has more boosts/weight. 

- **N-grams**: Sequences of adjacent words as terms. For example, since a word by itself may have little to no value, but if you were to put two words together and analyze it as a pair, then it might add more meaning. For example, "iPhone" VS "iPhone Charger"

- **Stemming and Lemmatization**: Get the root meaning of the word

- **Topic Models**: A type of model that represents a set of topics from a sequence of words. 

# Table of Contents

### Review
- [Core NLP Concepts](#correlation)

- [Feature Pre-Processing](#race_economic)

- [Feature Engineering](#school_attendance)

### Latent Dirichlet Allocation (LDA)
- [Topic Modeling](#definition)


### Model Selection:
- [Ridge, LASSO, LGBM](#correlation)

- [Cross Validation](#race_economic)

- [RMSLE Definition](#school_attendance)

- [Tree Based VS Non-Tree Based](#student_performance)

- [Ensemble Blend](#math_test)

### Eli5 
- [Evaluating Text Features](#math_test)

### Conclusion
- [Future Work](#correlation)

<img src='https://wrm5sysfkg-flywheel.netdna-ssl.com/wp-content/uploads/2018/07/Chilmark-Report_NLP-Technology-e1531398420983.png'>

***
# Review
Remember everything that we did from feature pre-processing, to transformations, and to derivations are important for text data because these will be our features for our model. You're going to have to experiment with stop-words, n-grams, and various other methods to come up with a good representation for your model.

## Text Feature Pre-Processing
- Normalizing Text
- Stemming Words
- Lemmitizing Words
- Remove Stop Words
- N-Grams

## Text Feature Transformations
- Bag of Words Model
- Log Transformation of Price
- Dummy (One Hot Encoding) of Categorical Variables
- Sparse Matrices to compress the data

## Text Feature Derivations
Generate corpus & Make TF-IDF Weights

### CountVectorizer 
Replaces tokens(words in this case) by their counts in the document.

["dog", "dog", "cat"] - > ["dog": 2, "cat": 1]

### TFIDF
Replaces tokens by the relative importance of the token in the document. 

### LabelBinarizer 
Converts tokens to binary vectors.

["tree", "human", "alien", "tree"] - > ["tree": [1, 0, 0], "human": [0, 1, 0], "alien": [0, 0, 1]]

***
# Training Set

In [1]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.sparse import csr_matrix, hstack
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer

from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import KFold, cross_val_score, train_test_split

from sklearn.linear_model import Ridge

## Get Training Data

In [42]:
# Get 10% of the Training Data
train = pd.read_csv('train.tsv', sep = '\t')
reduced_X_train = train.sample(frac=0.1).reset_index(drop=True)
reduced_y_train = np.log1p(reduced_X_train['price'])

## Fast Data Cleaning

In [3]:
# Fast Cleaning of Data
reduced_X_train['category_name'] = reduced_X_train['category_name'].fillna('Other').astype(str)
reduced_X_train['brand_name'] = reduced_X_train['brand_name'].fillna('missing').astype(str)
reduced_X_train['shipping'] = reduced_X_train['shipping'].astype(str)
reduced_X_train['item_condition_id'] = reduced_X_train['item_condition_id'].astype(str)
reduced_X_train['item_description'] = reduced_X_train['item_description'].fillna('None')

<img src='https://mith.umd.edu/wp-content/uploads/2015/10/header_topic-modeling.png'>

# Topic Modeling
***

### LDA – Latent Dirichlet Allocation 
Its foundations are Probabilistic Graphical Models


### Why is Topic Modeling Useful?
There are several scenarios when topic modeling can prove useful. Here are some of them:

1. Text classification – Topic modeling can improve classification by grouping similar words together in topics rather than using each word as a feature
2. Recommender Systems – Using a similarity measure we can build recommender systems. If our system would recommend articles for readers, it will recommend articles with a topic structure similar to the articles the user has already read.
3. Uncovering Themes in Texts – Useful for detecting trends in online publications for example

### How does LDA Work?
LDA is an iterative algorithm. Here are the two main steps:

1. In the initialization stage, each word is assigned to a random topic.
2. Iteratively, the algorithm goes through each word and reassigns the word to a topic taking into consideration: What’s the probability of the word belonging to a topic and What’s the probability of the document to be generated by a topic

Each topic in a document are percentages that all add up to 1.

Uses CV instead of TFIDF as weights


In [4]:
%%time
from sklearn.decomposition import LatentDirichletAllocation

# Initialize CountVectorizer
cvectorizer = CountVectorizer(max_features=20000,
                              stop_words='english', 
                              lowercase=True)

# Fit it to our dataset
cvz = cvectorizer.fit_transform(reduced_X_train['item_description'])

# Initialize LDA Model with 10 Topics
lda_model = LatentDirichletAllocation(n_components=10,
                                      random_state=42)

# Fit it to our CountVectorizer Transformation
X_topics = lda_model.fit_transform(cvz)

# Define variables
n_top_words = 10
topic_summaries = []

# Get the topic words
topic_word = lda_model.components_
# Get the vocabulary from the text features
vocab = cvectorizer.get_feature_names()

# Display the Topic Models
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
    topic_summaries.append(' '.join(topic_words))
    print('Topic {}: {}'.format(i, ' | '.join(topic_words)))

Topic 0: new | used | brand | box | color | matte | palette | lip | opened | like
Topic 1: rm | shipping | free | items | item | bundle | ship | know | new | purchase
Topic 2: bag | gold | new | silver | leather | condition | authentic | used | black | inside
Topic 3: new | brand | pink | size | tags | leggings | black | lularoe | free | color
Topic 4: skin | brush | oil | makeup | hair | use | rm | natural | face | note
Topic 5: size | condition | worn | small | new | great | good | black | medium | like
Topic 6: free | home | smoke | size | shipping | pet | bundle | inches | length | cotton
Topic 7: rm | disney | set | new | pink | stickers | bath | free | 10 | cards
Topic 8: price | shipping | rm | new | free | firm | bundle | brand | items | ship
Topic 9: description | iphone | case | condition | phone | works | great | comes | charger | game
Wall time: 12min 44s


***
# Eli5 


### Eli5  – Explain it Like I'm 5
- It's a library that allows you to see what your model has learned from the text features.
- Looking at features helps to **understand how your classifier works**. 


### Why is Eli5 Useful?
Looking at features helps to understand how classifier works. Maybe even more importantly, it helps to notice preprocessing bugs, data leaks, issues with task specification - all these nasty problems you get in a real world.

### How does Eli5 Work?
It shows you the correlation of each feature/text with the target variable. We can inspect features and weights because we’re using a bag-of-words vectorizer and a linear classifier (so there is a direct mapping between individual words and classifier coefficients). 

### Debugging Best Practices
- classifier assigns high weights to seemingly unrelated words like ‘do’ or ‘my’ -> Remove Stop Words

I think it might be more important to see what the model finds important and try to normalize maybe the top 10-30 tokens your particular model sees as important. Focusing on some of the top features and finding normalizations for that also mixed with some extra feature engineering has helped me push my score a little bit farther.

## Analyzing Item Description with Eli5

In [5]:
# Definte RMSLE Cross Validation Function
def rmsle_cv(model):
    kf = KFold(shuffle=True, random_state=42).get_n_splits(reduced_X_train['item_description'])
    rmse= np.sqrt(-cross_val_score(model, reduced_X_train['item_description'], reduced_y_train, scoring="neg_mean_squared_error", cv = kf))
    return(rmse.mean())

## Baseline Model with CountVectorizer

In [6]:
from sklearn.linear_model import Ridge

vec = CountVectorizer()
clf = Ridge(random_state=42)
pipe = make_pipeline(vec, clf)
pipe.fit(reduced_X_train['item_description'], reduced_y_train)

cv_rmsle = rmsle_cv(pipe)

print("The Validation Score is: " + str(cv_rmsle))

The Validation Score is: 0.6879862994784519


In [41]:
import eli5
eli5.show_weights(pipe, vec=vec, top=100, feature_filter=lambda x: x != '<BIAS>')

Weight?,Feature
+3.525,unlocked
+3.043,gb
+3.033,32gb
+3.008,14k
+2.700,receipt
+2.657,16gb
+2.631,dustbag
+2.385,imei
+2.356,lv
+2.329,date code


In [8]:
eli5.show_prediction(clf, doc=reduced_X_train['item_description'][1297], vec=vec)

## Baseline Model with CountVectorizer and Stop Words

In [9]:
vec = CountVectorizer(stop_words='english')
clf = Ridge(random_state=42)
pipe = make_pipeline(vec, clf)
pipe.fit(reduced_X_train['item_description'], reduced_y_train)

cv_sw_rmsle = rmsle_cv(pipe)

print("The Validation Score is: " + str(cv_sw_rmsle))

The Validation Score is: 0.6888015584524416


In [10]:
eli5.show_prediction(clf, doc=reduced_X_train['item_description'][1297], vec=vec)

## Baseline Model with TF-IDF

In [11]:
vec = TfidfVectorizer()
clf = Ridge(random_state=42)
pipe = make_pipeline(vec, clf)
pipe.fit(reduced_X_train['item_description'], reduced_y_train)

tfidf_rmsle = rmsle_cv(pipe)

print("The Validation Score is: " + str(tfidf_rmsle))

The Validation Score is: 0.6182196839738301


In [12]:
eli5.show_prediction(clf, doc=reduced_X_train['item_description'][1297], vec=vec)

## Baseline Model with TF-IDF and Stop Words

In [13]:
vec = TfidfVectorizer(stop_words='english')
clf = Ridge(random_state=42)
pipe = make_pipeline(vec, clf)
pipe.fit(reduced_X_train['item_description'], reduced_y_train)

tfidf_sw_rmsle = rmsle_cv(pipe)

print("The Validation Score is: " + str(tfidf_sw_rmsle))

The Validation Score is: 0.6201022810481083


In [14]:
eli5.show_prediction(clf, doc=reduced_X_train['item_description'][1297], vec=vec)

## Baseline Model with TF-IDF, Stop Words, and N-Grams

In [15]:
vec = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
clf = Ridge(random_state=42)
pipe = make_pipeline(vec, clf)
pipe.fit(reduced_X_train['item_description'], reduced_y_train)

tfidf_sw_ng_rmsle = rmsle_cv(pipe)

print("The Validation Score is: " + str(tfidf_sw_ng_rmsle))

The Validation Score is: 0.6060820485716552


In [16]:
eli5.show_prediction(clf, doc=reduced_X_train['item_description'][1297], vec=vec)

## RMSLE Summary

In [17]:
print ("RMSLE Score: " + str(cv_rmsle) + " | CountVectorizer")
print ("RMSLE Score: " + str(cv_sw_rmsle) + " | CountVectorizer | Stop Words")
print ("RMSLE Score: " + str(tfidf_rmsle) + " | TF-IDF")
print ("RMSLE Score: " + str(tfidf_sw_rmsle) + " | TF-IDF | Stop Words")
print ("RMSLE Score: " + str(tfidf_sw_ng_rmsle) + " | TF-IDF | Stop Words | N-Grams")

RMSLE Score: 0.6879862994784519 | CountVectorizer
RMSLE Score: 0.6888015584524416 | CountVectorizer | Stop Words
RMSLE Score: 0.6182196839738301 | TF-IDF
RMSLE Score: 0.6201022810481083 | TF-IDF | Stop Words
RMSLE Score: 0.6060820485716552 | TF-IDF | Stop Words | N-Grams


***
# Feature Pre-Processing / Transformation

It's super modular. So you have an estimator or a transformer, then you have a pipeline, then you connect more than one transformations together

In [18]:
from sklearn.pipeline import FeatureUnion

default_preprocessor = CountVectorizer().build_preprocessor()

def build_preprocessor(field):
    field_idx = list(reduced_X_train.columns).index(field)
    return lambda x: default_preprocessor(x[field_idx])

vectorizer = FeatureUnion([
    ('name', CountVectorizer(
        ngram_range=(1, 2),
        max_features=50000,
        preprocessor=build_preprocessor('name'))),
    ('category_name', CountVectorizer(
        token_pattern='.+',
        preprocessor=build_preprocessor('category_name'))),
    ('brand_name', CountVectorizer(
        token_pattern='.+',
        preprocessor=build_preprocessor('brand_name'))),
    ('shipping', CountVectorizer(
        token_pattern='\d+',
        preprocessor=build_preprocessor('shipping'))),
    ('item_condition_id', CountVectorizer(
        token_pattern='\d+',
        preprocessor=build_preprocessor('item_condition_id'))),
    ('item_description', TfidfVectorizer(
        ngram_range=(1, 2),
        max_features=55000,
        stop_words='english',
        preprocessor=build_preprocessor('item_description'))),
])

# Modeling

- Ridge Regression
- LASSO Regression
- Light GBM

### Create Transformed Training Set

In [19]:
# Create Transformed Train Set
reduced_Xt_train = vectorizer.fit_transform(reduced_X_train.values)
reduced_Xt_train

<148254x108418 sparse matrix of type '<class 'numpy.float64'>'
	with 4638517 stored elements in Compressed Sparse Row format>

### Define RMSLE Function

**Why RMSLE?**
- It puts more penalty on **lower errors**
- This is used when you want to penalize **under estimates** more than **over estimates**.

Lets have a look at the below example

Case a) : Pi = 600, Ai = 1000 RMSE = 400, RMSLE = 0.5108

Case b) : Pi = 1400, Ai = 1000 RMSE = 400, RMSLE = 0.3365

As it is evident, the differences are same between actual and predicted in both the cases. RMSE treated them equally however RMSLE penalized the under estimate more than over estimate.

In [20]:
def get_rmsle(y, pred): return np.sqrt(mean_squared_error(y, pred))

## Ridge Cross Validation

In [30]:
%%time

# Create 3-Fold CV
cv = KFold(n_splits=3, shuffle=True, random_state=42)
for train_ids, valid_ids in cv.split(reduced_Xt_train):
    # Define LGBM Model
    model_ridge = Ridge(solver = "auto", fit_intercept=True, random_state=42)
    
    # Fit LGBM Model
    model_ridge.fit(reduced_Xt_train[train_ids], reduced_y_train[train_ids])
    
    # Predict & Evaluate Training Score
    y_pred_train = model_ridge.predict(reduced_Xt_train[train_ids])
    rmsle_train = get_rmsle(y_pred_train, reduced_y_train[train_ids])
    
    # Predict & Evaluate Validation Score
    y_pred_valid = model_ridge.predict(reduced_Xt_train[valid_ids])
    rmsle_valid = get_rmsle(y_pred_valid, reduced_y_train[valid_ids])
    
    print(f'LGBM Training RMSLE: {rmsle_train:.5f}')
    print(f'LGBM Validation RMSLE: {rmsle_valid:.5f}')


LGBM Training RMSLE: 0.29585
LGBM Validation RMSLE: 0.52006
LGBM Training RMSLE: 0.29411
LGBM Validation RMSLE: 0.52278
LGBM Training RMSLE: 0.29440
LGBM Validation RMSLE: 0.52571
Wall time: 47.2 s


## LASSO Cross Validation

Why did LASSO Perform way worse than Ridge?
- Ridge RMSLE: 0.53 
- LASSO RMSLE: 0.74

One reason why could be because since LASSO performs automatic feature selection. So keep in mind majority of our features are just words. It'll remove some of our text features. And this may not generalize well with new data. Because our dataset is suppose to capture and use all our words as features. 

In [31]:
%%time
from sklearn.linear_model import Lasso

# Create 3-Fold CV
cv = KFold(n_splits=3, shuffle=True, random_state=42)
for train_ids, valid_ids in cv.split(reduced_Xt_train):
    # Define LGBM Model
    model_LASSO = Lasso(fit_intercept=True, random_state=42)
    
    # Fit LGBM Model
    model_LASSO.fit(reduced_Xt_train[train_ids], reduced_y_train[train_ids])
    
    # Predict & Evaluate Training Score
    y_pred_train = model_LASSO.predict(reduced_Xt_train[train_ids])
    rmsle_train = get_rmsle(y_pred_train, reduced_y_train[train_ids])
    
    # Predict & Evaluate Validation Score
    y_pred_valid = model_LASSO.predict(reduced_Xt_train[valid_ids])
    rmsle_valid = get_rmsle(y_pred_valid, reduced_y_train[valid_ids])
    
    print(f'LASSO Training RMSLE: {rmsle_train:.5f}')
    print(f'LASSO Validation RMSLE: {rmsle_valid:.5f}')


LASSO Training RMSLE: 0.75292
LASSO Validation RMSLE: 0.74664
LASSO Training RMSLE: 0.74933
LASSO Validation RMSLE: 0.75384
LASSO Training RMSLE: 0.75024
LASSO Validation RMSLE: 0.75202
Wall time: 2min 8s


## LGBM Cross Validation

**Reference:** https://medium.com/@pushkarmandot/https-medium-com-pushkarmandot-what-is-lightgbm-how-to-implement-it-how-to-fine-tune-the-parameters-60347819b7fc

Why LightGBM?
- ‘Light’ because of its high speed. 
- Can handle the large size of data
- Can take lower memory to run
- Focuses on accuracy of results

Why not LightGBM?
- When you have small data
- Prone to overfitting
- Needs a lot of tuning

How Gradient Boosting Trees work:
- Learn a regression predictor
- Compute the error residual
- Learn to predict the residual

Tree Based Models Pros:
- Automatically approximates linear transformations
- Automaically approximates subtle interactions between features (such as ratios/sums/differences of features)
- Gracefully handles missing values
- Deals with outliers well

In [23]:
%%time
import lightgbm as lgb

# Create 3-Fold CV
cv = KFold(n_splits=3, shuffle=True, random_state=42)
for train_ids, valid_ids in cv.split(reduced_Xt_train):
    # Define LGBM Model
    model_lgb = lgb.LGBMRegressor(num_leaves=31, n_jobs=-1, learning_rate=0.1, n_estimators=500, random_state=42)
    
    # Fit LGBM Model
    model_lgb.fit(reduced_Xt_train[train_ids], reduced_y_train[train_ids])
    
    # Predict & Evaluate Training Score
    y_pred_train = model_lgb.predict(reduced_Xt_train[train_ids])
    rmsle_train = get_rmsle(y_pred_train, reduced_y_train[train_ids])
    
    # Predict & Evaluate Validation Score
    y_pred_valid = model_lgb.predict(reduced_Xt_train[valid_ids])
    rmsle_valid = get_rmsle(y_pred_valid, reduced_y_train[valid_ids])
    
    print(f'LGBM Training RMSLE: {rmsle_train:.5f}')
    print(f'LGBM Validation RMSLE: {rmsle_valid:.5f}')
    

LGBM Training RMSLE: 0.46545
LGBM Validation RMSLE: 0.50923
LGBM Training RMSLE: 0.46232
LGBM Validation RMSLE: 0.51196
LGBM Training RMSLE: 0.46174
LGBM Validation RMSLE: 0.51270
Wall time: 8min 36s


<img src='https://image.slidesharecdn.com/ipbimprovingthemodelspredictivepowerwithensembleapproaches-121203224610-phpapp02/95/improving-the-models-predictive-power-with-ensemble-approaches-10-638.jpg?cb=1354575467'>

# Ensemble (Ridge + LGBM)

LGBM:
- Takes a while
- Starts with a weak learner and it is built upon each other to create more complex learners.

Ridge 
- Has a nice convex objective to maximise which is also easy to evaluate etc
- Unlike lasso or even elastic net it does not force parameters to be zero if their effect size is too small 
- Standard CountVectorizer features act mostly as dummies, and provided interactions between them are slight linear model should work quite well, as all you are mostly measuring is if the feature is on or off. 
- That's why better tokenisation / n-grams may improve the result, rather then attempting to capture "iPhone case" as interaction between iPhone and case keywords, it is better to capture "iPhone case" keyword directly.

Regularization:
- You can think of regularization as a technique that makes you fit your data less well so it can generalize it better on new data.
- Lasso does a variable selection, while Ridge does not
- When you have **highly-correlated variables**, **Ridge** shrinks the two coefficients. **Lasso** goes a step beyond that by setting them to zero and generally picks one over the other, and depending on the context you may not know which variable gets picked. **Elastic Net** does both: it shrinks and does sparse selection.

### Create Train/Test Split

In [32]:
# Train and Test Split
train_X, test_X, train_y, test_y = train_test_split(reduced_Xt_train, reduced_y_train, test_size=0.2, random_state=144)

### Define LGBM Model

In [34]:
# Define LGBM Model
model_lgb = lgb.LGBMRegressor(num_leaves=31, n_jobs=-1, learning_rate=0.1, n_estimators=500, random_state=42)

# Fit LGBM Model
model_lgb.fit(train_X, train_y)

# Predict with LGBM Model
lgbm_y_pred = model_lgb.predict(test_X)

### Define Ridge Model

In [35]:
# Define Ridge Model
model_ridge = Ridge(solver = "auto", fit_intercept=True, random_state=42)
    
# Fit Ridge Model
model_ridge.fit(train_X, train_y)
    
# Evaluate Training Score
ridge_y_pred = model_ridge.predict(test_X)

### Define Ensemble Model

**Definition::** Essentially an ensemble is where you get a bunch of indepdent models and you combine them toget one prediction.  
Tips for Ensemble:
- Start by guessing random weights. Start exploiting with a certain chance (say, 20% of the time) by nudging a weight up or down. Gradually increase exploitation rate.
- This should not be the case. Let's say you have a perfect and a lousy model, and you are trying to average them by finding optimal weights.  A weight-averaged model should always be at least as good as your best model, and in most cases it is better.
- Using different preprocessing methods like stemming/not stemming, different vectorizations, different tokenization can add diversity to the model, and may improve the quality of the ensemble. 

In [36]:
ensemble_y_pred = (lgbm_y_pred+ridge_y_pred)/2

ensemble_rmsle = get_rmsle(ensemble_y_pred, test_y)

print(f'Ensemble RMSLE: {ensemble_rmsle:.5f}')

Ensemble RMSLE: 0.48062


***
# Predictions

### Ensemble Predictions without Inverse Log Transformation

In [37]:
ensemble_y_pred[0:20]

array([2.64042419, 3.61434185, 2.27693474, 2.97912784, 2.99169578,
       3.02057209, 3.21861976, 3.0815065 , 3.53129627, 2.91790097,
       2.70583217, 2.76029778, 3.66321945, 2.59062505, 2.72804873,
       2.88163755, 2.18943772, 2.53466639, 2.08822952, 3.14119259])

### Ensemble Predictions (Inverse Log - Exponential)

In [38]:
# Esenmble Predictions
ensemble_y = (np.expm1(lgbm_y_pred)+np.expm1(ridge_y_pred))/2
ensemble_y[200:220]

array([25.55152166, 16.60230036, 10.97298503, 35.51587615, 33.77537553,
       17.27140988, 32.66264445, 12.69556525, 12.11171648,  9.9394762 ,
       14.25519241, 43.68088855, 17.22307723, 15.63676329, 14.14257929,
       11.04125465,  6.20560278, 19.33546376, 14.42017985, 43.28250821])

### Test Predictions (Inverse Log - Exponential)

In [39]:
# Test Predictions 
np.expm1(test_y[200:220])

27389     29.0
15132     10.0
48407      8.0
109582    32.0
21767     62.0
106154    32.0
4959      45.0
101231    14.0
102148     9.0
19708     10.0
127741     3.0
48785     35.0
133352    15.0
72243     18.0
28311     24.0
6026      18.0
21722      3.0
125264    36.0
52726     10.0
39941     31.0
Name: price, dtype: float64